## Práca s algoritmom K-Means

na tomto príklade si ukážeme prácu s algoritmom K-Means na jednoduchom príklade. 

Ukážeme si vytvorenie zhlukovacieho modelu na dátach, ktoré popisujú klientov veľkoobchodného predajcu. Pozostáva z informácií, ktoré charakterizujú ročné nákupy čerstvých, mliečnych, potravinových a iných produktov. Každý klient je popísaný nasledovnými atribútmi:
* Fresh - ročné výdajne na čerstvé produkty 
* Milk - ročné výdajne na mliečne produkty
* Grocery - ročné výdajne na potravinové produkty
* Frozen - ročné výdajne na mrazené produkty 
* Detergents_Paper - ročné výdajne na čistiace a papierové produkty
* Delicassen - ročné výdajne na delikatesy
* Channel - spôsob predaja zákazníkom hotnoty - Horeca (Hotel/Restaurant/Cafe) alebo Retail
* Region - nominálne hodnoty, zodpovedajú Lisbon, Porto alebo Other

Cieľom je vytvoriť zhlukovací model, ktorý by našiel v množine skupiny rôznych typov zákazníkov.

Najprv importujeme potrebné knižnice. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Načítame dáta zo súboru do dátového rámca a vypíšeme hlavičku. 

In [ ]:
data = pd.read_csv('../data/wholesale.csv')
data.head()

Keďže dataset obsahuje 2 kategorické atribúty (Channel a Region), tieto stĺpce transformujeme pomocou One Hot Encoder prístupu. 

In [ ]:
data = pd.get_dummies(data, columns=['Channel', 'Region']) 
data.head()

V prípade zhlukovania pomocou K-Means je veľmi podstatnou časťou výpočet vzdialeností jednotlivých príkladov od centier zhlukov. Aby sme zabezpečili rovnakú dôležitosť všetkých atribútov, tak ich normalizujeme na rovnaký rozsah. Podobne ako pri klasifikácii môžeme použiť `MinMaxScaler`, na celý dátový rámec (normalizujeme všetky atribúty).

Môžeme pretransformovať normalizované dáta do dátového rámca (ale nemusíme, ďalšie funkcie môžu pracovať s numpy poľom, ktoré dostaneme na výstupe normalizácie). 

In [ ]:
from sklearn.preprocessing import MinMaxScaler # importujeme MinMaxScaler

scaler = MinMaxScaler() # Inicializujeme transformátor
scaler.fit(data) # aplikujeme ho na vstupné dáta

# po aplikovaní scaleru budeme mať výstup vo forme numpy poľa
# to môžeme - ale nemusíme - naspať transformovať do pandas rámca (ak chceme ešte robiť nejaké predspracovanie)
# funkcie pre trénovanie modelov potom vedia pracovať aj s pandas aj s numpy

# data_norm = scaler.transform(data)
data_norm = pd.DataFrame(scaler.fit_transform(data), index=data.index, columns=data.columns)

Teraz vytvoríme K-Means zhlukovací model, ktorý natrénujeme na vstupných dátach. Implementácia K-Means v Sciki-learn umožňuje nasledovné nastavenia algoritmu (vybrali sme iba niektoré):
* n_clusters - zodpovedá hodnote parametra `k`, definuje počet zhlukov
* max_iter - zodpovedá maximálnemu počtu iterácií algoritmu (defaultná hodnota - 300)

Ako výstup po vytvorení modelu môžeme použiť:
* cluster_centers_ - pole súradníc centroidov pre jednotlivé zhluky
* labels_ - príslušnosť k zhlukom pre všetky objekty vstupných dát
* inertia_ - suma štvorcov vzdialeností príkladov k centroidu (kritérium)

In [ ]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=4)
model.fit(data_norm)
# labels = model.predict(data_norm)

Môžeme teraz vypísať napr. sumu štvorcov vzdialeností v rámci zhlukov pre vytvorený model alebo napr. spočítať vzdialenosti medzi jednotlivými centroidmi. 

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances # importujeme funkciu euclidean_distances ktorá nám spočíta vzdialenosti medzi zadanými bodmi

print("Inertia:") # zobrazíme vypočítanú inertiu
print(model.inertia_)

print("Vzajomne vzdialenosti centroidov:")
dists = euclidean_distances(model.cluster_centers_) # spočítame vzdialenosti medzi centrami zhlukov a vypíšeme ich
print(dists)

Takisto sa vieme pozrieť na obsah jednotlivých zhlukov. Pomocou `model.labels_` sa vieme pozrieť na to, do ktorých zhlukov model pridelil jednotlivé príklady zo vstupnej množiny dát. Môžeme sa takisto pozrieť na konkrétny zhluk a príklady, ktoré doň patria. 

In [ ]:
print("Príklady a ich príslušnosť do zhlukov:")
print(model.labels_) # vypíšeme príslušnost k zhluku pre každý príklad

In [ ]:
print("Príklady zo zhluku 0:")
cluster_0 = np.where(model.labels_==0) # vyberieme len príklady, ktoré patria do zhluku 0
print(cluster_0) # vypíšeme ich na obrazovku

Teraz si ukážeme ako môžeme porovnať dva zhluky navzájom. 

Vytvorím si teraz dátové rámce (nenormalizované) z príkladov pre jednotlivé zhluky. Teraz sa môžem pozrieť na obsah jednotlivých zhlukov a porovnať ich spoločné vlastnosti alebo rozdiely. 

In [ ]:
cluster_1 = np.where(model.labels_==1) # nájdeme príklady priradené do zhluku 1

data_cluster_0 = data.iloc[cluster_0] # dátový rámec z príkladov pre zhluk 0
data_cluster_1 = data.iloc[cluster_1] # dátový rámec z príkladov pre zhluk 1

In [ ]:
data_cluster_0.describe()

In [ ]:
data_cluster_1.describe()

Na porovnanie zhlukov môžeme robiť aj vizualizácie pomocou Seaborn.

### Úloha

Vizualizujte pomocou Seaborn napr.: 
* Distribúcie hodnôt zvolených atribútov pre jednotlivé zhluky
* Priemery hodnôt zvolených atribútov jednotlivých zhlukov  

In [ ]:
import seaborn as sns

# YOUR CODE HERE

Ako nájsť najlepšiu hodnotu počtu zhlukov? Môžeme použiť tzv. "elbow" metódu. Podobne ako pri Grid Search vytvoríme množinu modelov s rôznymi hodnotami parametra. Avšak v tomto prípade budeme potrebovať aj nejaké kritérium, ktoré by nám niečo povedalo o samotných zhlukoch. 

Keďže používame model K-Means, môžeme hľadať optimálny počet zhlukov tak, že budeme pre jednotlivé modely počítať sumu štvorcov vzdialeností príkladov zatriedených do zhluku k ich centroidu. Túto hodnotu vieme z modelu získať ako jeden z jeho 

In [ ]:
Sum_of_squared_distances = [] # prázdne pole stvorcov vzdialeností

K = range(1,15) # vygeneruje rozsah parametrov K

# v cykle vytboríme modely s rôznymy nastaveniami

for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(data_norm)
    Sum_of_squared_distances.append(km.inertia_)
    
print(Sum_of_squared_distances)

Môžeme vizializovať závislosť počtu zhlukov od ich "kompaktnosti".

In [ ]:
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('Pocet zhlukov')
plt.ylabel('Suma stvorcov vzdialenosti')
plt.title('Hladanie optimalneho poctu zhlukov')
plt.show()